# Binary Addition
> 8-bit Binary addition using Feed Forward Neural Networks. 

### Rules of Binary Addition
![](addition_table.jpg)

> Out network tries to learn these rules

### Feed Forward Neural Network for Binary Addition

![](FFNN.jpg)

In [38]:
import copy, numpy as np
np.random.seed(0)

### Required Functions for Implementing Neural Networks

In [39]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

In [40]:
def sigmoid_derivative(activation):
    return activation*(1-activation)

In [41]:
def add_binary_nums(x,y):
        x=bin(x)[2:]
        y=bin(y)[2:]
        max_len = max(len(x), len(y))
        flag = False
        x = x.zfill(max_len)
        y = y.zfill(max_len)

        result = ''
        carry = 0

        for i in range(max_len-1,-1,-1):
            r = carry
            r += 1 if x[i] == '1' else 0
            r += 1 if y[i] == '1' else 0
            result = ('1' if r % 2 == 1 else '0') + result
            carry = 0 if r < 2 else 1       

        if carry !=0 : 
            flag=True
            result = '1' + result

        return flag

### Creating a mapping of numbers and their binary equivalents

In [42]:
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = np.reshape(binary[i],(1,8))

### Hyperparameters

In [43]:
alpha=0.5
in_dim =16
hidden_dim = 48
out_dim = 8

### Our Weights

In [44]:
Wx = 2*np.random.random((in_dim,hidden_dim))-1
bx = 2*np.random.random((hidden_dim,1))-1
Wy = 2*np.random.random((hidden_dim, out_dim))-1
by = 2*np.random.random((out_dim,1))-1

### Preparing our data 

- Choose two random numbers 'a_int' and 'b_int'
- Find their binary representations
- Add them and find binary representation of the sum

**And our data is ready**

###### Now concatenate the binary representations and feed it to the network 

In [45]:

a_int = np.random.randint(largest_number/2)
a = int2binary[a_int] # binary encoding

b_int = np.random.randint(largest_number/2) 
b = int2binary[b_int]
ab = np.concatenate((a,b),axis=1)

c_int = a_int + b_int
c = int2binary[c_int]
pred = np.zeros_like(c)




### Forward Propagation

In [46]:
Zx = np.matmul(ab,Wx)+bx.T
Ax = 1/(1+np.exp(-Zx))
Zy = np.matmul(Ax,Wy)+by.T
Ay = 1/(1+np.exp(-Zy))

### Backpropagating the loss

In [47]:
loss = (c - Ay)**2
dAy = 2*(Ay-c)
dZy = dAy*Ay*(1-Ay)
dWy = np.matmul(Ax.T,dZy)
dBy = copy.deepcopy(dZy)
dAx = np.matmul(dZy,Wy.T)
dZx = dAx*Ax*(1-Ax)
dWx = ab.T*dZx
dBx = dZx

### Training the Network for 75000 samples

In [48]:
correct = 0
incorrect=0
has_carry = 0
alpha_arr = [1,1.0,1]
for k in range(len(alpha_arr)):
    np.random.seed(2)
    for j in range(25000):
        alpha = alpha_arr[k]
        a_int = np.random.randint(largest_number/2)
        a = int2binary[a_int]

        b_int = np.random.randint(largest_number/2)
        b = int2binary[b_int]
        ab = np.concatenate((a,b),axis=1)
        c_int = a_int + b_int
        c = int2binary[c_int]
        pred = np.zeros_like(c)
        Zx = np.matmul(ab,Wx)+bx.T
        Ax = 1/(1+np.exp(-Zx))
        Zy = np.matmul(Ax,Wy)+by.T
        Ay = 1/(1+np.exp(-Zy))
        loss = (c - Ay)**2
        dAy = 2*(Ay-c)
        dZy = dAy*Ay*(1-Ay)
        dWy = np.matmul(Ax.T,dZy)
        dBy = copy.deepcopy(dZy)
        dAx = np.matmul(dZy,Wy.T)
        dZx = dAx*Ax*(1-Ax)
        dWx = ab.T*dZx
        dBx = dZx
        Wx = Wx-(alpha*dWx)
        Wy = Wy-(alpha*dWy)
        bx = bx-(alpha*bx)
        by = by-(alpha*by)

        pred = np.copy(Ay)
        pred = np.round(pred)
        
        if np.array_equal(c,pred):
            correct += 1
        elif not np.array_equal(c,pred):
            incorrect +=1
            if add_binary_nums(a_int,b_int) :
                has_carry += 1
        

### Accuracy

In [49]:
(correct/75000)*100

65.78399999999999

### Number of samples that have an error and involve a carry term in their binary addition

In [50]:
has_carry

16665